# UK Traffic Accidents Severity Classification
4IZ565 – Programming for Data Science in Python

Spring Semester 2023/2024 Coursework

Authors: Terka Lukešová, Jan Štipl

## Dataset Description
The dataset contains statistics on traffic accidents in the UK, including many attributes, such as the route category and hazards, the vehicles involved, and some information about the drivers. Our goal will be to predict the target attribute Accident Severity, whose values are Slight/Serious/Fatal.

Dataset source: https://www.kaggle.com/datasets/tsiaras/uk-road-safety-accidents-and-vehicles

## Exploratory Data Analysis

### Initial settings

In [1]:
import random
import numpy as np
import sklearn
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sklearn.set_config(transform_output="pandas")

RANDOM_STATE = 42 # used throughout the notebook to make the results reproducable
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

### Load CSV tables and merge them 

In [2]:
# For speed during testing load only small percentage of the data
#p = 0.01  # 1% of the lines
#skip_rows = lambda i: i > 0 and random.random() > p
skip_rows = None

accidents_df = pd.read_csv("./data/Accident_Information.csv",
                           skiprows=skip_rows)
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 3169: invalid start byte
vehicle_df = pd.read_csv("./data/Vehicle_Information.csv", encoding="latin1")

/tmp/ipykernel_738135/3836563813.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents_df = pd.read_csv("./data/Accident_Information.csv",


In [3]:
df = pd.merge(accidents_df, vehicle_df, on="Accident_Index", how="inner")
del vehicle_df
del accidents_df

In [4]:
df

,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Control,Junction_Detail,Latitude,Light_Conditions,Local_Authority_(District),Local_Authority_(Highway),Location_Easting_OSGR,Location_Northing_OSGR,Longitude,LSOA_of_Accident_Location,Number_of_Casualties,Number_of_Vehicles,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Police_Force,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Year_x,InScotland,Age_Band_of_Driver,Age_of_Vehicle,Driver_Home_Area_Type,Driver_IMD_Decile,Engine_Capacity_.CC.,Hit_Object_in_Carriageway,Hit_Object_off_Carriageway,Journey_Purpose_of_Driver,Junction_Location,make,model,Propulsion_Code,Sex_of_Driver,Skidding_and_Overturning,Towing_and_Articulation,Vehicle_Leaving_Carriageway,Vehicle_Location.Restricted_Lane,Vehicle_Manoeuvre,Vehicle_Reference,Vehicle_Type,Was_Vehicle_Left_Hand_Drive,X1st_Point_of_Impact,Year_y
0,200501BS00002,B,450.000,C,0.000,Slight,None,2005-01-05,Wednesday,1.000,Auto traffic signal,Crossroads,51.520,Darkness - lights lit,Kensington and Chelsea,Kensington and Chelsea,524170.000,181650.000,-0.212,E01002909,1,1,0.000,5.000,Metropolitan Police,Dry,Dual carriageway,None,30.000,17:36,Urban,Fine no high winds,2005,No,36 - 45,3.000,Data missing or out of range,NaN,8268.000,None,None,Journey as part of work,Leaving roundabout,DENNIS,NaN,Heavy oil,Male,None,No tow/articulation,Did not leave carriageway,0.000,Slowing or stopping,1,Bus or coach (17 or more pass seats),No,Nearside,2005
1,200501BS00003,C,0.000,NaN,0.000,Slight,None,2005-01-06,Thursday,1.000,Data missing or out of range,Not at junction or within 20 metres,51.525,Darkness - lights lit,Kensington and Chelsea,Kensington and Chelsea,524520.000,182240.000,-0.206,E01002857,1,2,0.000,0.000,Metropolitan Police,Dry,Single carriageway,None,30.000,00:15,Urban,Fine no high winds,2005,No,26 - 35,5.000,Urban area,3.000,8300.000,Parked vehicle,None,Journey as part of work,Not at or within 20 metres of junction,DENNIS,NaN,Heavy oil,Male,None,No tow/articulation,Did not leave carriageway,0.000,Going ahead right-hand bend,1,Bus or coach (17 or more pass seats),No,Nearside,2005
2,200501BS00004,A,3220.000,NaN,0.000,Slight,None,2005-01-07,Friday,1.000,Data missing or out of range,Not at junction or within 20 metres,51.482,Daylight,Kensington and Chelsea,Kensington and Chelsea,526900.000,177530.000,-0.174,E01002840,1,1,0.000,0.000,Metropolitan Police,Dry,Single carriageway,None,30.000,10:35,Urban,Fine no high winds,2005,No,46 - 55,4.000,Urban area,1.000,1769.000,None,None,Other/Not known (2005-10),Not at or within 20 metres of junction,NISSAN,ALMERA SE AUTO,Petrol,Female,None,No tow/articulation,Did not leave carriageway,0.000,Going ahead other,1,Car,No,Front,2005
3,200501BS00005,Unclassified,0.000,NaN,0.000,Slight,None,2005-01-10,Monday,1.000,Data missing or out of range,Not at junction or within 20 metres,51.496,Darkness - lighting unknown,Kensington and Chelsea,Kensington and Chelsea,528060.000,179040.000,-0.157,E01002863,1,1,0.000,0.000,Metropolitan Police,Wet or damp,Single carriageway,None,30.000,21:13,Urban,Fine no high winds,2005,No,46 - 55,10.000,Data missing or out of range,NaN,85.000,Kerb,None,Other/Not known (2005-10),Not at or within 20 metres of junction,HONDA,NaN,Petrol,Male,Skidded,No tow/articulation,Did not leave carriageway,0.000,Going ahead other,1,Motorcycle 125cc and under,No,Front,2005
4,200501BS00006,Unclassified,0.000,NaN,0.000,Slight,None,2005-01-11,Tuesday,1.000,Data missing or out of range,Not at junction or within 20 metres,51.516,Daylight,Kensington and Chelsea,Kensington and Chelsea,524770.000,181160.000,-0.203,E01002832,1,2,0.000,0.000,Metropolitan Police,Wet or damp,Single carriageway,Oil or diesel,30.000,12:40,Urban,Raining no high winds,2005,No,46 - 55,1.000,Urban area,4.000,2976.000,None,

In [5]:
df.dtypes

Accident_Index                                  object
1st_Road_Class                                  object
1st_Road_Number                                float64
2nd_Road_Class                                  object
2nd_Road_Number                                float64
Accident_Severity                               object
Carriageway_Hazards                             object
Date                                            object
Day_of_Week                                     object
Did_Police_Officer_Attend_Scene_of_Accident    float64
Junction_Control                                object
Junction_Detail                                 object
Latitude                                       float64
Light_Conditions                                object
Local_Authority_(District)                      object
Local_Authority_(Highway)                       object
Location_Easting_OSGR                          float64
Location_Northing_OSGR                         float64
Longitude 

In [6]:
df.select_dtypes(include=['object']).nunique()

Accident_Index                 1389724
1st_Road_Class                       6
2nd_Road_Class                       6
Accident_Severity                    3
Carriageway_Hazards                  7
Date                              4383
Day_of_Week                          7
Junction_Control                     6
Junction_Detail                     10
Light_Conditions                     6
Local_Authority_(District)         416
Local_Authority_(Highway)          207
LSOA_of_Accident_Location        35475
Police_Force                        51
Road_Surface_Conditions              6
Road_Type                            7
Special_Conditions_at_Site           9
Time                              1439
Urban_or_Rural_Area                  3
Weather_Conditions                  10
InScotland                           2
Age_Band_of_Driver                  12
Driver_Home_Area_Type                4
Hit_Object_in_Carriageway           13
Hit_Object_off_Carriageway          13
Journey_Purpose_of_Driver

### Problems
* Unclassified and NaN in the same columns -> we have to merge
* Categorical values and not one-hot encoded

## Data preprocessing

### Separate the target class from attributes

In [7]:
target_variable = 'Accident_Severity'
X, y = df.loc[:, df.columns != target_variable], df[target_variable]
del df

### Consolidate string values Unknown/Unclassified to NaN

In [8]:
X.replace(
    [
        "Not known",
        "Data missing or out of range",
        "Unclassified"
    ],
    np.nan, inplace=True)
X.fillna(np.nan, inplace=True)

/tmp/ipykernel_738135/792606326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace(
/tmp/ipykernel_738135/792606326.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(np.nan, inplace=True)


### Train-test split
Our train-test ratio is 80:20. We make use of stratification because the target class is imbalanced.

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RANDOM_STATE)
del X
del y

### Drop columns with too many missing values
The columns are identified on the train set, the drop is then applied to the test set too

In [10]:
# Get percentage of NaN values for each column
na_percentage = X_train.isna().mean() * 100
na_percentage.sort_values(ascending=False, inplace=True)
na_percentage

2nd_Road_Class                                80.315
Journey_Purpose_of_Driver                     41.271
Junction_Control                              36.329
Driver_IMD_Decile                             33.491
1st_Road_Class                                26.302
Age_of_Vehicle                                16.416
Driver_Home_Area_Type                         14.630
model                                         14.545
Engine_Capacity_.CC.                          12.161
Propulsion_Code                               11.349
Age_Band_of_Driver                             7.900
LSOA_of_Accident_Location                      6.766
make                                           5.384
Sex_of_Driver                                  3.547
2nd_Road_Number                                0.922
Was_Vehicle_Left_Hand_Drive                    0.444
Junction_Location                              0.185
Road_Surface_Conditions                        0.173
X1st_Point_of_Impact                          

In [11]:
to_drop = na_percentage[na_percentage > 40]
to_drop

2nd_Road_Class              80.315
Journey_Purpose_of_Driver   41.271
dtype: float64

In [12]:
X_train.drop(columns=to_drop.index, inplace=True)
X_test.drop(columns=to_drop.index, inplace=True)

### Transformers

In [13]:
# Extract month from date
month_extractor = lambda x: pd.to_datetime(x['Date'], format='%Y-%m-%d').dt.month
X_train['month'] = month_extractor(X_train)
X_test['month'] = month_extractor(X_test)

# Extract hour from time
hour_extractor = lambda x: pd.to_datetime(x['Time'], format='%H:%M').dt.hour
X_train['hour'] = month_extractor(X_train)
X_test['hour'] = month_extractor(X_test)

In [14]:
numerical_variables = [
     # 'Number_of_Casualties', # Leaks info about accident severity
     # 'Number_of_Vehicles', # Leaks info about accident severity
    'Speed_limit',
    'Age_of_Vehicle',
    'Engine_Capacity_.CC.',
]

nominal_variables = [
    'Day_of_Week',
    'month',
    'hour',
    'Carriageway_Hazards',
    'Junction_Detail',
    'Pedestrian_Crossing-Human_Control',
    'Pedestrian_Crossing-Physical_Facilities',
    'Road_Type',
    'Special_Conditions_at_Site',
    'Urban_or_Rural_Area',
    'Weather_Conditions',
    'Driver_Home_Area_Type',
    'Hit_Object_in_Carriageway',
    'Hit_Object_off_Carriageway',
    'Junction_Location',
    'make',
    'Propulsion_Code',
    'Sex_of_Driver',
    'Skidding_and_Overturning',
    'Towing_and_Articulation',
    'Vehicle_Leaving_Carriageway',
    'Vehicle_Location.Restricted_Lane',
    'Vehicle_Manoeuvre',
    'Vehicle_Type',
    'Was_Vehicle_Left_Hand_Drive',
    'X1st_Point_of_Impact',
]

# Order of the values can be determined lexicographically
ordinal_variables_auto = [
    'Age_Band_of_Driver',
    'Driver_IMD_Decile',
]

In [15]:
# Correct order must be set manually
ordinal_variables_manual = [
    '1st_Road_Class',
    'Junction_Control',
    'Light_Conditions',
    'Road_Surface_Conditions',
]

road_class_categories = ['Motorway', 'A(M)', 'A', 'B', 'C']

junction_control_categories = [
    'Not at junction or within 20 metres',
    'Authorised person',
    'Auto traffic signal',
    'Stop sign',
    'Give way or uncontrolled',
]

light_conditions_categories = [
    'Daylight',
    'Darkness - lights lit',
    'Darkness - lighting unknown',
    'Darkness - lights unlit',
    'Darkness - no lighting',
]

road_surface_conditions_categories = [
    'Dry',
    'Wet or damp',
    'Snow',
    'Frost or ice',
    'Flood over 3cm. deep',
]

manual_categories = [
    road_class_categories,
    junction_control_categories,
    light_conditions_categories,
    road_surface_conditions_categories,
]

In [16]:
X_train.filter(nominal_variables + ordinal_variables_auto + ordinal_variables_manual).nunique()

Day_of_Week                                  7
month                                       12
hour                                        12
Carriageway_Hazards                          6
Junction_Detail                              9
Pedestrian_Crossing-Human_Control            3
Pedestrian_Crossing-Physical_Facilities      6
Road_Type                                    6
Special_Conditions_at_Site                   8
Urban_or_Rural_Area                          3
Weather_Conditions                           9
Driver_Home_Area_Type                        3
Hit_Object_in_Carriageway                   12
Hit_Object_off_Carriageway                  12
Junction_Location                            9
make                                       515
Propulsion_Code                             12
Sex_of_Driver                                2
Skidding_and_Overturning                     6
Towing_and_Articulation                      6
Vehicle_Leaving_Carriageway                  9
Vehicle_Locat

In [17]:
from sklearn.compose import ColumnTransformer  # applies transformers to columns
from sklearn.pipeline import Pipeline  # assemble several steps
from sklearn.impute import SimpleImputer  # replace missing values using a descriptive statistic (e.g. mean, median,...)
from sklearn.preprocessing import OrdinalEncoder  # encode categorical features as an integer array
from sklearn.preprocessing import OneHotEncoder  # encode categorical features as a one-hot numeric array
from sklearn.preprocessing import StandardScaler  # standardize features by removing the mean and scaling to unit variance

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(drop='if_binary', sparse_output=False)),
])

ord_pipe_auto = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OrdinalEncoder()),
])

ord_pipe_manual = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OrdinalEncoder(categories=manual_categories)),
])

transformers_list = [
    ('numerical', num_pipe, numerical_variables),
    ('nominal', cat_pipe, nominal_variables),
    ('ordinal_auto', ord_pipe_auto, ordinal_variables_auto),
    ('ordinal_manual', ord_pipe_manual, ordinal_variables_manual),
]

column_transformer = ColumnTransformer(transformers_list,
                                       remainder='drop',
# If True, ColumnTransformer.get_feature_names_out will prefix all feature names with the name of the transformer that generated that feature.
                                       verbose_feature_names_out=False,
                                       verbose=True)

X_train = column_transformer.fit_transform(X_train)
X_train.head()

[ColumnTransformer] ..... (1 of 4) Processing numerical, total=   0.1s
[ColumnTransformer] ....... (2 of 4) Processing nominal, total=  20.7s
[ColumnTransformer] .. (3 of 4) Processing ordinal_auto, total=   1.4s
[ColumnTransformer]  (4 of 4) Processing ordinal_manual, total=   1.9s


,Speed_limit,Age_of_Vehicle,Engine_Capacity_.CC.,Day_of_Week_Friday,Day_of_Week_Monday,Day_of_Week_Saturday,Day_of_Week_Sunday,Day_of_Week_Thursday,Day_of_Week_Tuesday,Day_of_Week_Wednesday,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,Carriageway_Hazards_Any animal in carriageway (except ridden horse),Carriageway_Hazards_None,Carriageway_Hazards_Other object on road,Carriageway_Hazards_Pedestrian in carriageway - not injured,Carriageway_Hazards_Previous accident,Carriageway_Hazards_Vehicle load on road,Junction_Detail_Crossroads,Junction_Detail_Mini-roundabout,Junction_Detail_More than 4 arms (not roundabout),Junction_Detail_Not at junction or within 20 metres,Junction_Detail_Other junction,Junction_Detail_Private drive or entrance,Junction_Detail_Roundabout,Junction_Detail_Slip road,Junction_Detail_T or staggered junction,Pedestrian_Crossing-Human_Control_0.0,Pedestrian_Crossing-Human_Control_1.0,Pedestrian_Crossing-Human_Control_2.0,Pedestrian_Crossing-Physical_Facilities_0.0,Pedestrian_Crossing-Physical_Facilities_1.0,Pedestrian_Crossing-Physical_Facilities_4.0,Pedestrian_Crossing-Physical_Facilities_5.0,Pedestrian_Crossing-Physical_Facilities_7.0,Pedestrian_Crossing-Physical_Facilities_8.0,Road_Type_Dual carriageway,Road_Type_One way street,Road_Type_Roundabout,Road_Type_Single carriageway,Road_Type_Slip road,Road_Type_Unknown,Special_Conditions_at_Site_Auto signal part defective,Special_Conditions_at_Site_Auto traffic signal - out,Special_Conditions_at_Site_Mud,Special_Conditions_at_Site_None,Special_Conditions_at_Site_Oil or diesel,Special_Conditions_at_Site_Road sign or marking defective or obscured,Special_Conditions_at_Site_Road surface defective,Special_Conditions_at_Site_Roadworks,Urban_or_Rural_Area_Rural,Urban_or_Rural_Area_Unallocated,Urban_or_Rural_Area_Urban,Weather_Conditions_Fine + high winds,Weather_Conditions_Fine no high winds,Weather_Conditions_Fog or mist,Weather_Conditions_Other,Weather_Conditions_Raining + high winds,Weather_Conditions_Raining no high winds,Weather_Conditions_Snowing + high winds,Weather_Conditions_Snowing no high winds,Weather_Conditions_Unknown,Driver_Home_Area_Type_Rural,Driver_Home_Area_Type_Small town,Driver_Home_Area_Type_Urban area,Hit_Object_in_Carriageway_Any animal (except ridden horse),Hit_Object_in_Carriageway_Bollard or refuge,Hit_Object_in_Carriageway_Bridge (roof),Hit_Object_in_Carriageway_Bridge (side),Hit_Object_in_Carriageway_Central island of roundabout,Hit_Object_in_Carriageway_Kerb,Hit_Object_in_Carriageway_None,Hit_Object_in_Carriageway_Open door of vehicle,Hit_Object_in_Carriageway_Other object,Hit_Object_in_Carriageway_Parked vehicle,Hit_Object_in_Carriageway_Previous accident,Hit_Object_in_Carriageway_Road works,Hit_Object_off_Carriageway_Bus stop or bus shelter,Hit_Object_off_Carriageway_Central crash barrier,Hit_Object_off_Carriageway_Entered ditch,Hit_Object_off_Carriageway_Lamp post,Hit_Object_off_Carriageway_Near/Offside crash barrier,Hit_Object_off_Carriageway_None,Hit_Object_off_Carriageway_Other permanent object,Hit_Object_off_Carriageway_Road sign or traffic signal,Hit_Object_off_Carriageway_Submerged in water,Hit_Object_off_Carriageway_Telegraph or electricity pole,Hit_Object_off_Carriageway_Tree,Hit_Object_off_Carriageway_Wall or fence,Junction_Location_Approaching junction or waiting/parked at junction approach,Junction_Location_Cleared junction or waiting/parked at junction exit,Junction_Location_Entering from slip road,Junction_Location_Entering main road,Junction_Location_Entering roundabout,Junction_Location_Leaving main road,Junction_Location_Leaving roundabout,Junction_Location_Mid Junction - on roundabout or on main road,Junction_Location_Not at or within 20 metres of junction,make_ABARTH,make_AC (ELECTRIC),make_ACCESS,make_ACURA,make_ADLY,make_AEBI,make_AEON,make_AGRIFAC,make_AIXAM,make_AJP,make_AJS,mak

In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
label_encoder.classes_

array(['Fatal', 'Serious', 'Slight'], dtype=object)

### Check correct order of ordinal categories

In [19]:
column_transformer.named_transformers_['ordinal_auto'].named_steps['encode'].categories_

[array(['0 - 5', '11 - 15', '16 - 20', '21 - 25', '26 - 35', '36 - 45',
        '46 - 55', '56 - 65', '6 - 10', '66 - 75', 'Over 75'], dtype=object),
 array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0], dtype=object)]

In [20]:
column_transformer.named_transformers_['ordinal_manual'].named_steps['encode'].categories_

[array(['Motorway', 'A(M)', 'A', 'B', 'C'], dtype=object),
 array(['Not at junction or within 20 metres', 'Authorised person',
        'Auto traffic signal', 'Stop sign', 'Give way or uncontrolled'],
       dtype=object),
 array(['Daylight', 'Darkness - lights lit', 'Darkness - lighting unknown',
        'Darkness - lights unlit', 'Darkness - no lighting'], dtype=object),
 array(['Dry', 'Wet or damp', 'Snow', 'Frost or ice',
        'Flood over 3cm. deep'], dtype=object)]

## Modeling

## Results and Evaluation